### Namaste! This notebook demonstrates question answering on Hindi data using Indic LLM - Airavata, a multilingual embedding model and Chroma db

Lets get started by collecting dataset, if you already have the dataset, parsed and prepared, you can skip through this part. We will be taking 5 URLs related to income tax, the url comprises of faq as well as unstructured text. The topics discussed include various sections for deduction on tax, various faqs related to ITR1 for indiviudals and various forms required.

In [1]:
urls =['https://www.incometax.gov.in/iec/foportal/hi/help/e-filing-itr1-form-sahaj-faq',
        'https://www.incometax.gov.in/iec/foportal/hi/help/e-filing-itr4-form-sugam-faq',
       'https://navbharattimes.indiatimes.com/business/budget/budget-classroom/income-tax-sections-know-which-section-can-save-how-much-tax-here-is-all-about-income-tax-law-to-understand-budget-speech/articleshow/89141099.cms',
       'https://www.incometax.gov.in/iec/foportal/hi/help/individual/return-applicable-1',
       'https://www.zeebiz.com/hindi/personal-finance/income-tax/tax-deductions-under-section-80g-income-tax-exemption-limit-how-to-save-tax-on-donation-money-to-charitable-trusts-126529'
]


I will be using one of my favorite libraries to crawl website - [Markdown Crawler](https://github.com/paulpierre/markdown-crawler). You can install it using the command mentioned below. It parses the website into markdown format and stores them in markdown files. ANd one interesting thing, although we will be crawling only the urls, but it also has the capability to parse linked urls to website(look at the depth paramaneter).
For now lets continue with what we were doing!

In [3]:
!pip install markdown-crawler
!pip install markdownify

In [4]:
from markdown_crawler import md_crawl
def crawl_urls(urls: list, storage_folder_path: str, max_depth=0):
    """Crawl a list of URLs and store results.
    Parameters:
    - urls: URLs to crawl.
    - storage_folder_path: Location for results; folder is auto-created.
    - max_depth: Link depth to crawl; 0 (recommended) means only the listed URLs.

    """
    for url in urls:
        print(f"Crawling {url}")
        md_crawl(
            url,
            max_depth=max_depth,
            base_dir=storage_folder_path,
            is_links=True
        )

In [5]:
crawl_urls(urls= urls, storage_folder_path = './incometax_documents/')
#you do not need to make a folder intitially. Md Crawler handles that for you.

Crawling https://www.incometax.gov.in/iec/foportal/hi/help/e-filing-itr1-form-sahaj-faq
Crawling https://www.incometax.gov.in/iec/foportal/hi/help/e-filing-itr4-form-sugam-faq
Crawling https://navbharattimes.indiatimes.com/business/budget/budget-classroom/income-tax-sections-know-which-section-can-save-how-much-tax-here-is-all-about-income-tax-law-to-understand-budget-speech/articleshow/89141099.cms
Crawling https://www.incometax.gov.in/iec/foportal/hi/help/individual/return-applicable-1
Crawling https://www.zeebiz.com/hindi/personal-finance/income-tax/tax-deductions-under-section-80g-income-tax-exemption-limit-how-to-save-tax-on-donation-money-to-charitable-trusts-126529


Once the urls have been crawled and data has been stored in markdown files, its time to parse the content in those files. But before that some notes on parsing:


It is important to parse, because there is still some noise left in the markdown files, and the embedding model that we are gonna use and llm has token limit, so we have to not exceed that.

Before we move on to the code, some parameters that I need to introduce. The embedding model that we are gonna use has a token limit of 512 tokens. It truncates after that. We will look more about this embedding model and why I chose this in the coming section but for now only thing we have to know is that the limit is 512 tokens so we will try to keep section less than 512 tokens.

Lets first write a function to extract content out of a file. We will be use python library markdown and beautifulsoup for it. Below are commands to install them

In [6]:
!pip install beautifulsoup4
!pip install markdown

In [7]:
# lets first write a function to extract content out of a file
import markdown
from bs4 import BeautifulSoup

def read_markdown_file(file_path):
    """Read a Markdown file and extract its sections as headers and content.

    Args:
    - file_path: Path to the Markdown file.

    Returns:
    - List of sections, each containing a header and corresponding content.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        md_content = file.read()

    html_content = markdown.markdown(md_content)
    soup = BeautifulSoup(html_content, 'html.parser')

    sections = []
    current_section = None

    for tag in soup:
        if tag.name and tag.name.startswith('h'):
            if current_section:
                sections.append(current_section)
            current_section = {'header': tag.text, 'content': ''}
        elif current_section:
            current_section['content'] += tag.get_text() + '\n'

    if current_section:
        sections.append(current_section)

    return sections

In [8]:
#lets look at the output of one of the files:

sections = read_markdown_file('./incometax_documents/business-budget-budget-classroom-income-tax-sections-know-which-section-can-save-how-much-tax-here-is-all-about-income-tax-law-to-understand-budget-speech-articleshow-89141099-cms.md')

Oh yes! the content looks cleaner now, but the problem is sometimes the section are good,and sometiems they are note, some sections with especially with empty headers are unnecessary. So lets write a funcito to parse a section, we will pass  a particular section only if the header and content both are non empty and header does not belong to any of these - ['main navigation','navigation', 'footer'].

In [9]:
def pass_section(section):
  headers_to_ignore = ['main navigation','navigation', 'footer','advertisement']
  if section['header'].lower() not in headers_to_ignore and section['header'].strip()and section['content'].strip():
    return True
  return False

In [10]:
passed_sections = []
import os
# Iterate through all Markdown files in the folder
for filename in os.listdir('incometax_documents'):
    if filename.endswith('.md'):
        file_path = os.path.join('incometax_documents', filename)
        # Extract sections from the current Markdown file
        all_sections = read_markdown_file(file_path)
        # Filter sections based on the pass_section function
        passed_sections.extend(section for section in all_sections if pass_section(section))

We will use the above function later. But before that lets talk about one of the most important aspect i.e chunking.

Chunking - Why do we chunk?
So every RAG uses atleast these two major models - 'Embedding Model' for embedding generation which is used for retrieval and 'Language Model' which is used for "Answer generation". Both of these models have their own limit on the number of input tokens which they can take. So it is recommended that you should consider that limit which divinding unstructured content into various sections.

The embedding model that we are going to use has limit of 512 tokens.


So what is the limit in our case.
To keep things simple for now, lets only talk about the limit of embedding model as the Language Model that we are going to use has limit greater than the sentence embedding model so it will be taken care of.

So what is the limit of embedding model?
But but but before that what is emebdding model.
The embedding model that we are gonna use is [multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base).  As mentioned on its huggingface page it supports 100 languages, although low-resource languages may see performance degradation But i have observed it performs fairly decent for Hindi. If you want better accuracy you can use try [BGE M3](https://huggingface.co/BAAI/bge-m3) as well but that is pretty resource intesive. Also OpenAI embeddings might perform well here, but lets stick to everything opensource for now. Hence a light weight but decent model. E5 it is.

Coming back to the limit part - It has a limit of 512 tokens. Alright we will keep that in mind. Moving further, lets first initiate the sentence transformer model , as we will need to calculate the .




In [11]:
!pip install chromadb

In [12]:
import chromadb
chroma_client = chromadb.Client()

In [13]:
from chromadb.utils import embedding_functions
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-base")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
collection = chroma_client.create_collection(name="income_tax_hindi", embedding_function= sentence_transformer_ef, metadata={"hnsw:space": "cosine"})

In [15]:
# chroma_client.delete_collection(name="income_tax_hindi")

In [16]:
collection.add(
    documents=[section['content'] for section in passed_sections],
    metadatas = [{'header': section['header']} for section in passed_sections],
    ids=[str(i) for i in range(len(passed_sections))]
)

In [17]:
docs = collection.query(
    query_texts=["सेक्शन 80 C की लिमिट क्या होती है"],
    n_results=3
)

In [18]:
docs

{'ids': [['15', '17', '11']],
 'embeddings': None,
 'documents': [['\n\nसेक्शन 80डी के अलावा इनकम टैक्स कानून में दो और सेक्शन हैं, जिनका आप स्वास्थ्य से जुड़े खर्च का लाभ उठा सकते हैं। सेक्शन 80डीडी आप पर आश्रित किसी विकलांग व्यक्ति के लिए चिकित्सा खर्च से संबंधित है। आश्रित में जीवनसाथी, बच्चे, पैरेंट्स, भाई या बहन हो सकते हैं। इनकम टैक्स में छूट इस बात पर निर्भर करता है कि आपके आश्रित की विकलांगता कितनी गंभीर है। अगर आश्रित 40 फीसदी तक विकलांग है तो टैक्स बचत के लिए 75,000 रुपये तक का मेडिकल खर्च कवर किया जा सकता है। अगर आश्रित 80 फीसदी तक विकलांग है तो टैक्स बचत के लिए 1,25,000 रुपये तक का मेडिकल खर्च कवर किया जा सकता है।\n\n\n',
   '\n\nअगर आप खुद 40 फीसदी से अधिक विकलांग हैं तो आप इस सेक्शन के तहत Income Tax छूट पा सकते हैं। हालांकि, सेक्शन 80यू और सेक्शन 80डीडी का लाभ एक साथ नहीं उठाया जा सकता। इस सेक्शन में भी टैक्स छूट का लाभ सेक्शन 80डीडी की तरह ही होता है। फर्क सिर्फ इतना है कि यह सेक्शन खुद की विकलांगता से जुड़ा है, जबकि सेक्शन 80डी आश्रितों से जुड़ा है।\n\n\n',
   '\n\nसामा

In [19]:
docs = [doc for doc in docs['documents'][0]]
docs = "\n".join(docs)

In [20]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [36]:
def create_prompt_with_chat_format(messages, bos="<s>", eos="</s>", add_bos=True):
    formatted_text = ""
    for message in messages:
        if message["role"] == "system":
            formatted_text += "<|system|>\n" + message["content"] + "\n"
        elif message["role"] == "user":
            formatted_text += "<|user|>\n" + message["content"] + "\n"
        elif message["role"] == "assistant":
            formatted_text += "<|assistant|>\n" + message["content"].strip() + eos + "\n"
        else:
            raise ValueError(
                "Tulu chat template only supports 'system', 'user' and 'assistant' roles. Invalid role: {}.".format(
                    message["role"]
                )
            )
    formatted_text += "<|assistant|>\n"
    formatted_text = bos + formatted_text if add_bos else formatted_text
    return formatted_text


def inference(input_prompts, model, tokenizer):
    input_prompts = [
        create_prompt_with_chat_format([{"role": "user", "content": input_prompt}], add_bos=False)
        for input_prompt in input_prompts
    ]

    encodings = tokenizer(input_prompts, padding=True, return_tensors="pt")
    encodings = encodings.to(device)

    with torch.inference_mode():
        outputs = model.generate(encodings.input_ids, do_sample=False, max_new_tokens=100)

    output_texts = tokenizer.batch_decode(outputs.detach(), skip_special_tokens=True)

    input_prompts = [
        tokenizer.decode(tokenizer.encode(input_prompt), skip_special_tokens=True) for input_prompt in input_prompts
    ]
    output_texts = [output_text[len(input_prompt) :] for input_prompt, output_text in zip(input_prompts, output_texts)]
    return output_texts

In [23]:
model_name = "ai4bharat/Airavata"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name,  quantization_config=quantization_config, torch_dtype=torch.bfloat16)

# input_prompts = [
#     "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ? मुझे पांच बिंदु बताएं।",
#     "मैं अपने समय प्रबंधन कौशल को कैसे सुधार सकता हूँ? मुझे पांच बिंदु बताएं और उनका वर्णन करें।",
# ]
# outputs = inference(input_prompts, model, tokenizer)
# print(outputs)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [24]:
prompt =f'''आप एक बड़े भाषा मॉडल हैं जो दिए गए संदर्भ के आधार पर सवालों का उत्तर देते हैं। नीचे दिए गए निर्देशों का पालन करें:

1. **प्रश्न पढ़ें**:
    - दिए गए सवाल को ध्यान से पढ़ें और समझें।

2. **संदर्भ पढ़ें**:
    - नीचे दिए गए संदर्भ को ध्यानपूर्वक पढ़ें और समझें।

3. **सूचना उत्पन्न करना**:
    - संदर्भ का उपयोग करते हुए, प्रश्न का विस्तृत और स्पष्ट उत्तर तैयार करें।
    - यह सुनिश्चित करें कि उत्तर सीधा, समझने में आसान और तथ्यों पर आधारित हो।

### उदाहरण:

**संदर्भ**:
    "नई दिल्ली भारत की राजधानी है और यह देश का प्रमुख राजनीतिक और प्रशासनिक केंद्र है। यह शहर ऐतिहासिक स्मारकों, संग्रहालयों और विविध संस्कृति के लिए जाना जाता है।"

**प्रश्न**:
    "भारत की राजधानी क्या है और यह क्यों महत्वपूर्ण है?"

**प्रत्याशित उत्तर**:
    "भारत की राजधानी नई दिल्ली है। यह देश का प्रमुख राजनीतिक और प्रशासनिक केंद्र है और ऐतिहासिक स्मारकों, संग्रहालयों और विविध संस्कृति के लिए जाना जाता है।"

### निर्देश:

अब, दिए गए संदर्भ और प्रश्न का उपयोग करके उत्तर दें:

**संदर्भ**:
{docs}



**प्रश्न**:
    ""kya सेक्शन 80यू और सेक्शन 80डीडी का लाभ एक साथ उठाया जा सकता ""

उत्तर देने के लिए उपरोक्त निर्देशों का पालन करें और सुनिश्चित करें कि उत्तर सटीक और स्पष्ट हो।
'''

In [38]:
prompt ='''आप एक बड़े भाषा मॉडल हैं जो दिए गए संदर्भ के आधार पर सवालों का उत्तर देते हैं। नीचे दिए गए निर्देशों का पालन करें:

1. **प्रश्न पढ़ें**:
    - दिए गए सवाल को ध्यान से पढ़ें और समझें।

2. **संदर्भ पढ़ें**:
    - नीचे दिए गए संदर्भ को ध्यानपूर्वक पढ़ें और समझें।

3. **सूचना उत्पन्न करना**:
    - संदर्भ का उपयोग करते हुए, प्रश्न का विस्तृत और स्पष्ट उत्तर तैयार करें।
    - यह सुनिश्चित करें कि उत्तर सीधा, समझने में आसान और तथ्यों पर आधारित हो।

### उदाहरण:

**संदर्भ**:
    "नई दिल्ली भारत की राजधानी है और यह देश का प्रमुख राजनीतिक और प्रशासनिक केंद्र है। यह शहर ऐतिहासिक स्मारकों, संग्रहालयों और विविध संस्कृति के लिए जाना जाता है।"

**प्रश्न**:
    "भारत की राजधानी क्या है और यह क्यों महत्वपूर्ण है?"

**प्रत्याशित उत्तर**:
    "भारत की राजधानी नई दिल्ली है। यह देश का प्रमुख राजनीतिक और प्रशासनिक केंद्र है और ऐतिहासिक स्मारकों, संग्रहालयों और विविध संस्कृति के लिए जाना जाता है।"

### निर्देश:

अब, दिए गए संदर्भ और प्रश्न का उपयोग करके उत्तर दें:

**संदर्भ**:
{docs}

**प्रश्न**:
{query}

उत्तर:

'''
def generate_answer(query):
  docs =  collection.query(
    query_texts=[query],
    n_results=3
)
  docs = [doc for doc in docs['documents'][0]]
  docs = "\n".join(docs)
  formatted_prompt = prompt.format(docs = docs,query = query)
  answers = inference([formatted_prompt], model, tokenizer)
  return answers[0]



In [40]:
questions = [
    'सेक्शन 80डीडी के तहत विकलांग आश्रित के लिए कौन से मेडिकल खर्च पर टैक्स छूट मिल सकती है?',
    'क्या सेक्शन 80यू और सेक्शन 80डीडी का लाभ एक साथ उठाया जा सकता है?',
    'सेक्शन 80 C की लिमिट क्या होती है?'
]

for question in questions:
    answer = generate_answer(question)
    print(f"Question: {question}\nAnswer: {answer}\n")

Question: सेक्शन 80डीडी के तहत विकलांग आश्रित के लिए कौन से मेडिकल खर्च पर टैक्स छूट मिल सकती है?
Answer: आश्रित के लिए टैक्स छूट उन खर्चों पर उपलब्ध है जो 40 फीसदी से अधिक विकलांगता वाले व्यक्ति के लिए आवश्यक हैं। इन खर्चों में अस्पताल में भर्ती होना, सर्जरी, दवाएं और चिकित्सा उपकरण शामिल हैं।

Question: क्या सेक्शन 80यू और सेक्शन 80डीडी का लाभ एक साथ उठाया जा सकता है?
Answer: नहीं।

Question: सेक्शन 80 C की लिमिट क्या होती है?
Answer: सेक्शन 80सी की सीमा 1.5 लाख रुपये है।



In [ ]:
from google.colab import drive
drive.mount('/content/drive')